In [2]:
import pandas as pd             # data package
import matplotlib.pyplot as plt # graphics 
import datetime as dt
import numpy as np
from census import Census # This is new...

import requests, io             # internet and input tools  
import zipfile as zf            # zip file tools 
import os  

#import weightedcalcs as wc
#import numpy as np

import pyarrow as pa
import pyarrow.parquet as pq

In [3]:
file = ".\\data"+ "\\phaseone-tradedata-C-april.parquet"

df = pq.read_table(file).to_pandas()

df.replace({'Electrical equipment and mac.hioery':'Electrical equipment and machinery',
           'Industrial mac.hin ery': 'Industrial machinery'}, inplace = True)

df["hs2"] = df.hs4.str[0:2]

In [4]:
df.tail()

,time,CTY_CODE,china_exports,hs6,china_exports_quant,quant_type,hs4,hs4_o,description,low_catagory,high_catagory,_merge,hs2
549,2021-04-01,None,301693665.0,None,NaN,None,9018,9018,"Instruments and appliances used in medic,al su...",Optical and medical instruments,Manufactures,None,90
550,2021-04-01,None,11836447.0,None,NaN,None,9019,9019,Mechano-therapy appliances; massage apparatus;...,Optical and medical instruments,Manufactures,None,90
551,2021-04-01,None,2968130.0,None,NaN,None,9020,9020,"Other breathing appliances and gas masks, excl...",Optical and medical instruments,Manufactures,None,90
552,2021-04-01,None,72038540.0,None,NaN,None,9021,9021,"Orthopedic appliances, including crutch,es su...",Optical and medical instruments,Manufactures,None,90
553,2021-04-01,None,64583312.0,None,NaN,None,9022,9022,Apparatus based on the use of X-rays or of alp...,Optical and medical instruments,Manufactures,None,90


In [5]:
df = df[df["high_catagory"] != "not in aggreement"]

In [6]:
my_key = "&key=34e40301bda77077e24c859c6c6c0b721ad73fc7"

end_use = "hs?get=E_COMMODITY,ALL_VAL_YR,E_COMMODITY_SDESC"

url = "https://api.census.gov/data/timeseries/intltrade/exports/" + end_use 
url = url + my_key + "&time=2019" + "&COMM_LVL=HS2"

url = url + "&CTY_CODE=5700"

r = requests.get(url) 
    
print(r)
    
dfc = pd.DataFrame(r.json()[1:]) # This then converts it to a dataframe
    # Note that the first entry is the labels

dfc.columns = r.json()[0]

dfc = dfc.groupby("E_COMMODITY").agg({"E_COMMODITY_SDESC":"first"})

<Response [200]>


In [7]:
dfc.head()

,E_COMMODITY_SDESC
E_COMMODITY,
01,LIVE ANIMALS
02,MEAT AND EDIBLE MEAT OFFAL
03,"FISH, CRUSTACEANS & AQUATIC INVERTEBRATES"
04,DAIRY PRODS; BIRDS EGGS; HONEY; ED ANIMAL PR N...
05,"PRODUCTS OF ANIMAL ORIGIN, NESOI"


In [8]:
df.head()

,time,CTY_CODE,china_exports,hs6,china_exports_quant,quant_type,hs4,hs4_o,description,low_catagory,high_catagory,_merge,hs2
0,2013-01-01,5700,4806.0,841090,0.0,-,8410,8410,"Hydraulic turbines, water wheels and regulator...",Industrial machinery,Manufactures,both,84
11,2013-01-01,5700,279107.0,900211,0.0,-,9002,9002,"Lens,es prism, s mirrors and other optical e...",Optical and medical instruments,Manufactures,both,90
26,2013-01-01,5700,4007.0,110630,0.0,-,1106,1106,Flour and meal of dried leguminous vegetables ...,Cereals,Agriculture,both,11
27,2013-01-01,5700,404107.0,130212,0.0,-,1302,1302,Vegetable saps and extracts; pectic substances...,Other agricultural commodities,Agriculture,both,13
28,2013-01-01,5700,438996.0,841191,0.0,-,8411,8411,"Turboj e,ts turbopropellers and other gas ...",Industrial machinery,Manufactures,both,84


In [9]:
dfhs2 = df.groupby(["hs2","time"]).agg({"china_exports":"sum"})

In [10]:
dfhs2 = dfhs2.merge(dfc, left_on = ["hs2"], right_index = True, how = "left")

dfhs2.reset_index(inplace = True)

dfhs2["E_COMMODITY_SDESC"] = "HS CODE " + dfhs2.hs2 + ", " + dfhs2["E_COMMODITY_SDESC"]

In [11]:
dfhs2.set_index(["E_COMMODITY_SDESC","time"],inplace = True)

In [12]:
dfhs2.rename({"hs2":"hs-code"},axis = 1, inplace = True)

In [13]:
dfhs2

hs-code  china_exports
E_COMMODITY_SDESC                        time                             
HS CODE 01, LIVE ANIMALS                 2013-01-01      01      7136452.0
                                         2013-02-01      01      2969368.0
                                         2013-03-01      01      7543461.0
                                         2013-04-01      01      1601596.0
                                         2013-05-01      01      7695188.0
...                                                     ...            ...
HS CODE 90, OPTIC, PHOTO ETC, MEDIC OR S 2020-12-01      90    477988759.0
                                         2021-01-01      90    395318968.0
                                         2021-02-01      90    346107868.0
                                         2021-03-01      90    557599547.0
                                         2021-04-01      90    462538432.0

[4511 rows x 2 columns]

In [14]:
file = ".\\data"+ "\\phaseone-tradedata-C-april.parquet"

df = pq.read_table(file).to_pandas()

df.replace({'Electrical equipment and mac.hioery':'Electrical equipment and machinery',
           'Industrial mac.hin ery': 'Industrial machinery'}, inplace = True)

df["hs2"] = df.hs4.str[0:2]

df = df[df["high_catagory"] != "not in aggreement"]

dfhs4 = df.groupby(["description","time"]).agg({"china_exports":"sum", "hs4": "first"})

dfhs4.reset_index(inplace = True)

dfhs4["description"] = "HS CODE " + dfhs4["hs4"] + ", " + dfhs4["description"]

dfhs4.set_index(["description", "time"], inplace = True)

In [15]:
dfhs4.rename({"hs4":"hs-code"},axis = 1, inplace = True)

In [16]:
dfhs4

china_exports  \
description                                        time                        
HS CODE 2911, Acetals and hemiacetals, whether ... 2013-01-01        76799.0   
                                                   2013-02-01        67971.0   
                                                   2013-03-01        30507.0   
                                                   2013-04-01            0.0   
                                                   2013-05-01        83827.0   
...                                                                      ...   
HS CODE 2817, Zinc oxide; zinc peroxide            2020-12-01        36671.0   
                                                   2021-01-01        19993.0   
                                                   2021-02-01       143273.0   
                                                   2021-03-01       275107.0   
                                                   2021-04-01       160330.0   

                                                              hs-code  
description                                        time                
HS CODE 2911, Acetals and hemiacetals, whether ... 2013-01-01    2911  
                                                   2013-02-01    2911  
                                                   2013-03-01    2911  
                                                   2013-04-01    2911  
                                                   2013-05-01    2911  
...                                                               ...  
HS CODE 2817, Zinc oxide; zinc peroxide            2020-12-01    2817  
                                                   2021-01-01    2817  
                                                   2021-02-01    2817  
                                                   2021-03-01    2817  
                                                   2021-04-01    2817  

[47539 rows x 2 columns]

In [17]:
dfall = dfhs2.append(dfhs4)

In [18]:
test = dfall.index.unique(0).to_list()

In [19]:
dfall.loc["HS CODE 72, IRON AND STEEL"]

,hs-code,china_exports
time,,
2013-01-01,72,114103112.0
2013-02-01,72,158804662.0
2013-03-01,72,128598274.0
2013-04-01,72,98804647.0
2013-05-01,72,146780917.0
...,...,...
2020-12-01,72,30452723.0
2021-01-01,72,24959941.0
2021-02-01,72,23251669.0


In [20]:
out_file = ".\\data"+ "\\data.parquet"

pq.write_table(pa.Table.from_pandas(dfall), out_file)

In [21]:
file = ".\\data"+ "\\data.parquet"

df = pq.read_table(file).to_pandas()

df.loc["HS CODE 72, IRON AND STEEL"]

,hs-code,china_exports
time,,
2013-01-01,72,114103112.0
2013-02-01,72,158804662.0
2013-03-01,72,128598274.0
2013-04-01,72,98804647.0
2013-05-01,72,146780917.0
...,...,...
2020-12-01,72,30452723.0
2021-01-01,72,24959941.0
2021-02-01,72,23251669.0


In [20]:
df.china_exports.max()

3557751166.0

In [21]:
df[df.hs2 == "31"]

,,hs2,china_exports
E_COMMODITY_SDESC,time,,


In [22]:
df.tail()

hs-code  china_exports
E_COMMODITY_SDESC                       time                             
HS CODE 2817, Zinc oxide; zinc peroxide 2020-11-01    2817       123218.0
                                        2020-12-01    2817        36671.0
                                        2021-01-01    2817        19993.0
                                        2021-02-01    2817       143273.0
                                        2021-03-01    2817       275107.0